In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns 
import matplotlib.pyplot as plt
from datasist.structdata import detect_outliers 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb
import pickle

In [5]:
sns.set(rc={"figure.figsize": [9,9]}, font_scale=1.5)

In [6]:
df=pd.read_csv(r"C:\Users\omara\Desktop local\career\internship\cellula\week 4\final_internship_data.csv")

In [7]:
df.head()

,User ID,User Name,Driver Name,Car Condition,Weather,Traffic Condition,key,fare_amount,pickup_datetime,pickup_longitude,...,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing
0,KHVrEVlD,Kimberly Adams,Amy Butler,Very Good,windy,Congested Traffic,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-1.288826,...,6,0,2009,20.265840,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897
1,lPxIuEri,Justin Tapia,Hannah Zimmerman,Excellent,cloudy,Flow Traffic,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-1.291824,...,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217
2,gsVN8JLS,Elizabeth Lopez,Amanda Jackson,Bad,stormy,Congested Traffic,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-1.291242,...,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961
3,9I7kWFgd,Steven Wilson,Amy Horn,Very Good,stormy,Flow Traffic,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-1.291319,...,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.799270,0.133905
4,8QN5ZaGN,Alexander Andrews,Cassandra Larson,Bad,stormy,Congested Traffic,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-1.290987,...,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703


In [8]:
df.columns

Index(['User ID', 'User Name', 'Driver Name', 'Car Condition', 'Weather',
       'Traffic Condition', 'key', 'fare_amount', 'pickup_datetime',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'hour', 'day', 'month',
       'weekday', 'year', 'jfk_dist', 'ewr_dist', 'lga_dist', 'sol_dist',
       'nyc_dist', 'distance', 'bearing'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   User ID            500000 non-null  object 
 1   User Name          500000 non-null  object 
 2   Driver Name        500000 non-null  object 
 3   Car Condition      500000 non-null  object 
 4   Weather            500000 non-null  object 
 5   Traffic Condition  500000 non-null  object 
 6   key                500000 non-null  object 
 7   fare_amount        500000 non-null  float64
 8   pickup_datetime    500000 non-null  object 
 9   pickup_longitude   500000 non-null  float64
 10  pickup_latitude    500000 non-null  float64
 11  dropoff_longitude  499995 non-null  float64
 12  dropoff_latitude   499995 non-null  float64
 13  passenger_count    500000 non-null  int64  
 14  hour               500000 non-null  int64  
 15  day                500000 non-null  int64  
 16  mo

In [10]:
df.isna().sum()

User ID              0
User Name            0
Driver Name          0
Car Condition        0
Weather              0
Traffic Condition    0
key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    5
dropoff_latitude     5
passenger_count      0
hour                 0
day                  0
month                0
weekday              0
year                 0
jfk_dist             5
ewr_dist             5
lga_dist             5
sol_dist             5
nyc_dist             5
distance             5
bearing              5
dtype: int64

In [11]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing
count,500000.000000,500000.000000,500000.000000,499995.000000,499995.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,499995.000000,499995.000000,499995.000000,499995.000000,499995.000000,499995.000000,499995.000000
mean,11.358361,-1.265712,0.696740,-1.265755,0.696675,1.683428,13.510834,15.684206,6.268650,3.042008,2011.739132,385.279367,380.503657,363.843772,363.674038,355.991423,19.468775,0.297145
std,9.916617,0.206941,0.140909,0.205903,0.128997,1.307395,6.511571,8.681066,3.437815,1.949240,1.860889,2419.087483,2428.804740,2425.075903,2428.348683,2428.730839,367.299601,1.804548
min,-44.900000,-52.119764,-54.389440,-59.049665,-44.676047,0.000000,0.000000,1.000000,1.000000,0.000000,2009.000000,1.017646,1.460945,0.382119,0.532545,0.080500,0.000000,-3.141593
25%,6.000000,-1.291405,0.710958,-1.291393,0.710943,1.000000,9.000000,8.000000,3.000000,1.000000,2010.000000,41.341514,32.173712,17.100762,14.886989,7.147384,1.214550,-0.854721
50%,8.500000,-1.291226,0.711268,-1.291197,0.711277,1.000000,14.000000,16.000000,6.000000,3.000000,2012.000000,42.523163,34.787507,19.591554,18.347580,10.458151,2.116970,-0.050442
75%,12.500000,-1.290970,0.711520,-1.290908,0.711538,2.000000,19.000000,23.000000,9.000000,5.000000,2013.000000,43.785649,38.304502,22.214815,22.417812,14.448699,3.890070,2.206769
max,500.000000,37.360538,29.724576,0.712985,7.061893,6.000000,23.000000,31.000000,12.000000,6.000000,2015.000000,30133.067880,30167.595967,30167.285794,30159.407296,30162.285356,12399.956433,3.141593


In [12]:
df.duplicated().sum()

0

In [13]:
df.sample(3)

,User ID,User Name,Driver Name,Car Condition,Weather,Traffic Condition,key,fare_amount,pickup_datetime,pickup_longitude,...,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing
271113,GJ8o01LW,James Davis,Christopher Dunn,Very Good,rainy,Flow Traffic,2012-01-26 20:28:00.000000240,6.5,2012-01-26 20:28:00,-1.291044,...,1,3,2012,43.621992,35.667422,18.219860,19.973691,12.054204,1.484644,1.101867
455314,ijlHKtEv,Brian Hernandez,Willie Pollard,Bad,windy,Congested Traffic,2012-06-28 23:11:00.000000111,5.3,2012-06-28 23:11:00,-1.291495,...,6,3,2012,41.861373,28.363041,25.696578,9.456898,1.724182,1.648639,2.367440
92078,GCFC9wW3,Mr. Edward Powell,Brittany Baxter,Good,cloudy,Flow Traffic,2013-04-11 14:24:00.00000022,19.5,2013-04-11 14:24:00,-1.290751,...,4,3,2013,43.748416,37.290036,17.427910,21.469782,13.443987,5.493693,2.570645


In [14]:
df.dropna(axis=0,inplace=True)

In [15]:
df.isna().sum()

User ID              0
User Name            0
Driver Name          0
Car Condition        0
Weather              0
Traffic Condition    0
key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
hour                 0
day                  0
month                0
weekday              0
year                 0
jfk_dist             0
ewr_dist             0
lga_dist             0
sol_dist             0
nyc_dist             0
distance             0
bearing              0
dtype: int64

In [16]:
df.columns

Index(['User ID', 'User Name', 'Driver Name', 'Car Condition', 'Weather',
       'Traffic Condition', 'key', 'fare_amount', 'pickup_datetime',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'hour', 'day', 'month',
       'weekday', 'year', 'jfk_dist', 'ewr_dist', 'lga_dist', 'sol_dist',
       'nyc_dist', 'distance', 'bearing'],
      dtype='object')

In [17]:
df.drop(["key","Driver Name","User Name","User ID"] , axis=1,inplace=True)


In [18]:
df.head()

,Car Condition,Weather,Traffic Condition,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,...,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing
0,Very Good,windy,Congested Traffic,4.5,2009-06-15 17:26:21,-1.288826,0.710721,-1.288779,0.710563,1,...,6,0,2009,20.265840,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897
1,Excellent,cloudy,Flow Traffic,16.9,2010-01-05 16:52:16,-1.291824,0.710546,-1.291182,0.711780,1,...,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217
2,Bad,stormy,Congested Traffic,5.7,2011-08-18 00:35:00,-1.291242,0.711418,-1.291391,0.711231,2,...,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961
3,Very Good,stormy,Flow Traffic,7.7,2012-04-21 04:30:42,-1.291319,0.710927,-1.291396,0.711363,1,...,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.799270,0.133905
4,Bad,stormy,Congested Traffic,5.3,2010-03-09 07:51:00,-1.290987,0.711536,-1.290787,0.711811,1,...,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703


In [19]:
def haversine (lon_1, lon_2, lat_1, lat_2):
    
    lon_1, lon_2, lat_1, lat_2 = map(np.radians, [lon_1, lon_2, lat_1, lat_2])  
    diff_lon = lon_2 - lon_1
    diff_lat = lat_2 - lat_1
    km = 2 * 6371 * np.arcsin(np.sqrt(np.sin(diff_lat/2.0)**2 + 
                                      np.cos(lat_1) * np.cos(lat_2) * np.sin(diff_lon/2.0)**2))
    return km

In [20]:
df["my distance"] =haversine(df["pickup_longitude"],df["dropoff_longitude"],df["pickup_latitude"],df["dropoff_latitude"])

In [21]:
df.head()

,Car Condition,Weather,Traffic Condition,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,...,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing,my distance
0,Very Good,windy,Congested Traffic,4.5,2009-06-15 17:26:21,-1.288826,0.710721,-1.288779,0.710563,1,...,0,2009,20.265840,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897,0.018312
1,Excellent,cloudy,Flow Traffic,16.9,2010-01-05 16:52:16,-1.291824,0.710546,-1.291182,0.711780,1,...,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217,0.154664
2,Bad,stormy,Congested Traffic,5.7,2011-08-18 00:35:00,-1.291242,0.711418,-1.291391,0.711231,2,...,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961,0.026537
3,Very Good,stormy,Flow Traffic,7.7,2012-04-21 04:30:42,-1.291319,0.710927,-1.291396,0.711363,1,...,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.799270,0.133905,0.049179
4,Bad,stormy,Congested Traffic,5.3,2010-03-09 07:51:00,-1.290987,0.711536,-1.290787,0.711811,1,...,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703,0.037784


In [22]:
df.columns

Index(['Car Condition', 'Weather', 'Traffic Condition', 'fare_amount',
       'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'hour',
       'day', 'month', 'weekday', 'year', 'jfk_dist', 'ewr_dist', 'lga_dist',
       'sol_dist', 'nyc_dist', 'distance', 'bearing', 'my distance'],
      dtype='object')

In [23]:
df.drop(["dropoff_latitude","dropoff_longitude","pickup_latitude","pickup_longitude"] , axis=1,inplace=True)

In [24]:
df.head()

,Car Condition,Weather,Traffic Condition,fare_amount,pickup_datetime,passenger_count,hour,day,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing,my distance
0,Very Good,windy,Congested Traffic,4.5,2009-06-15 17:26:21,1,17,15,6,0,2009,20.265840,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897,0.018312
1,Excellent,cloudy,Flow Traffic,16.9,2010-01-05 16:52:16,1,16,5,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217,0.154664
2,Bad,stormy,Congested Traffic,5.7,2011-08-18 00:35:00,2,0,18,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961,0.026537
3,Very Good,stormy,Flow Traffic,7.7,2012-04-21 04:30:42,1,4,21,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.799270,0.133905,0.049179
4,Bad,stormy,Congested Traffic,5.3,2010-03-09 07:51:00,1,7,9,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703,0.037784


In [25]:
df["pickup_datetime"]=pd.to_datetime(df["pickup_datetime"],format="%Y-%m-%d %H:%M:%S",errors='coerce')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499995 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Car Condition      499995 non-null  object        
 1   Weather            499995 non-null  object        
 2   Traffic Condition  499995 non-null  object        
 3   fare_amount        499995 non-null  float64       
 4   pickup_datetime    499995 non-null  datetime64[ns]
 5   passenger_count    499995 non-null  int64         
 6   hour               499995 non-null  int64         
 7   day                499995 non-null  int64         
 8   month              499995 non-null  int64         
 9   weekday            499995 non-null  int64         
 10  year               499995 non-null  int64         
 11  jfk_dist           499995 non-null  float64       
 12  ewr_dist           499995 non-null  float64       
 13  lga_dist           499995 non-null  float64      

In [27]:
df=df.convert_dtypes()

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499995 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Car Condition      499995 non-null  string        
 1   Weather            499995 non-null  string        
 2   Traffic Condition  499995 non-null  string        
 3   fare_amount        499995 non-null  Float64       
 4   pickup_datetime    499995 non-null  datetime64[ns]
 5   passenger_count    499995 non-null  Int64         
 6   hour               499995 non-null  Int64         
 7   day                499995 non-null  Int64         
 8   month              499995 non-null  Int64         
 9   weekday            499995 non-null  Int64         
 10  year               499995 non-null  Int64         
 11  jfk_dist           499995 non-null  Float64       
 12  ewr_dist           499995 non-null  Float64       
 13  lga_dist           499995 non-null  Float64      

In [29]:
df.isna().sum()

Car Condition        0
Weather              0
Traffic Condition    0
fare_amount          0
pickup_datetime      0
passenger_count      0
hour                 0
day                  0
month                0
weekday              0
year                 0
jfk_dist             0
ewr_dist             0
lga_dist             0
sol_dist             0
nyc_dist             0
distance             0
bearing              0
my distance          0
dtype: int64

In [30]:
df["min"]=df["pickup_datetime"].dt.minute

In [31]:
df.head()

,Car Condition,Weather,Traffic Condition,fare_amount,pickup_datetime,passenger_count,hour,day,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing,my distance,min
0,Very Good,windy,Congested Traffic,4.5,2009-06-15 17:26:21,1,17,15,6,0,2009,20.26584,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897,0.018312,26
1,Excellent,cloudy,Flow Traffic,16.9,2010-01-05 16:52:16,1,16,5,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217,0.154664,52
2,Bad,stormy,Congested Traffic,5.7,2011-08-18 00:35:00,2,0,18,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961,0.026537,35
3,Very Good,stormy,Flow Traffic,7.7,2012-04-21 04:30:42,1,4,21,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.79927,0.133905,0.049179,30
4,Bad,stormy,Congested Traffic,5.3,2010-03-09 07:51:00,1,7,9,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703,0.037784,51


In [32]:
df.drop(["pickup_datetime"],axis=1,inplace=True)

In [33]:
df.head()

,Car Condition,Weather,Traffic Condition,fare_amount,passenger_count,hour,day,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing,my distance,min
0,Very Good,windy,Congested Traffic,4.5,1,17,15,6,0,2009,20.26584,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897,0.018312,26
1,Excellent,cloudy,Flow Traffic,16.9,1,16,5,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217,0.154664,52
2,Bad,stormy,Congested Traffic,5.7,2,0,18,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961,0.026537,35
3,Very Good,stormy,Flow Traffic,7.7,1,4,21,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.79927,0.133905,0.049179,30
4,Bad,stormy,Congested Traffic,5.3,1,7,9,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703,0.037784,51


In [ ]:
px.pie(df,names=df["Car Condition"],title= "Car condition distribution").update_traces(textinfo="percent")

In [ ]:
px.histogram(df, x="Car Condition", y="fare_amount", histfunc="avg", title="Average Fare Amount by Car Condition")

In [ ]:
px.histogram(df, x="Car Condition", y=df["distance"], histfunc="avg", title="Average distance by Car Condition")

In [ ]:
px.histogram(df, x="Car Condition", y=df["my distance"], histfunc="avg", title="Average distance by Car Condition")

In [ ]:
df["Car Condition"].unique()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 5), sharey=True)
conditions = ['Very Good', 'Excellent', 'Bad', 'Good']

for i, condition in enumerate(conditions):
    sns.boxplot(data=df[df['Car Condition'] == condition],
                y='fare_amount',
                ax=axes[i],
                color='skyblue')
    axes[i].set_title(f"Car Condition: {condition}")
    axes[i].set_xlabel("")  
    axes[i].set_ylabel("Fare Amount ($)" if i == 0 else "") 

plt.tight_layout()
plt.suptitle("Fare Amount Distribution by Car Condition", fontsize=16, y=1.05)
plt.show()


In [ ]:
px.histogram(df, x="Car Condition", y=df["passenger_count"], histfunc="avg", title="Average passenger count by Car Condition")

In [ ]:
df["Weather"].unique()

In [ ]:
px.pie(df,names=df["Weather"],title= "Weather condition distribution").update_traces(textinfo="percent")

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=df, x="Car Condition", hue="Weather")
plt.title("Weather Distribution by Car Condition")
plt.ylabel("Number of Rides")
plt.xticks(rotation=30)
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
sns.boxplot(data=df, x="Car Condition", y="fare_amount", hue="Weather")
plt.title("Fare Amount Distribution by Car Condition and Weather")
plt.ylabel("Fare Amount")
plt.xticks(rotation=15)
plt.legend(title="Weather", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()

In [ ]:
df["Traffic Condition"].unique()

In [ ]:
px.pie(df,names=df["Traffic Condition"],title= "Traffic condition distribution").update_traces(textinfo="percent")

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=df, x="Car Condition", hue="Traffic Condition")
plt.title("Traffic Condition Distribution by Car Condition")
plt.ylabel("Number of Rides")
plt.xticks(rotation=30)
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
sns.boxplot(data=df, x="Car Condition", y="fare_amount", hue=df["Traffic Condition"])
plt.title("Fare Amount Distribution by Car Condition and Traffic Condition")
plt.ylabel("Fare Amount")
plt.xticks(rotation=15)
plt.legend(title="Traffic Condition", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()

In [ ]:
df.head()

In [ ]:
px.box(df, x="Weather", y="fare_amount", color="Car Condition",title="Fare Amount Distribution by Weather and Car Condition")


In [ ]:
px.box(df, x=df["Traffic Condition"], y="fare_amount", color="Car Condition",title="Fare Amount Distribution by Traffic Condition and Car Condition")


In [ ]:
df.head()

In [ ]:
sns.scatterplot(df,x=df["passenger_count"],y=df["fare_amount"])
plt.title("Passenger Count By Fare Amount")

In [ ]:
sns.scatterplot(df,x=df["distance"],y=df["fare_amount"])
plt.title("distance By Fare Amount")

In [ ]:
sns.scatterplot(df,x=df["my distance"],y=df["fare_amount"])
plt.title("my distance By Fare Amount")

In [ ]:
df[df["fare_amount"]<=0].count()

In [ ]:
df.drop(df[df["fare_amount"]<=0].index,axis=0,inplace=True)

In [ ]:
df[df["distance"]<=0]

In [ ]:
df.info()

In [ ]:
df.drop(df[df["distance"]<=0].index,axis=0,inplace=True)

In [ ]:
df.info()

In [ ]:
px.scatter(df,x=df["distance"],y=df["fare_amount"],title="Fare Amount By Distance ")

In [ ]:
sns.scatterplot(df,x=df["distance"],y=df["fare_amount"])
plt.title("Fare Amount By Distance ")

In [ ]:
sns.scatterplot(df,x=df["my distance"],y=df["fare_amount"])
plt.title("Fare Amount By Distance ")

In [ ]:
df[df["my distance"]<=0].count()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".1f", cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
px.box(df,x=df["distance"])

In [ ]:
px.box(df,x=df["my distance"])

In [ ]:
sns.boxplot(df,x=df["fare_amount"])

In [ ]:
df[(df["fare_amount"]>60) & (df["distance"]<1) ]

In [ ]:
df.describe()

In [ ]:
df[df["fare_amount"]==df["fare_amount"].min()]

In [ ]:
sns.histplot(df,x=df["fare_amount"])

In [ ]:
df[df["fare_amount"]==df["fare_amount"].min()]

In [ ]:
df.drop(df[df["fare_amount"]==df["fare_amount"].min()].index, axis=0,inplace=True)

In [ ]:
df[df["fare_amount"]==df["fare_amount"].min()]

In [ ]:
df[df["distance"]==df["distance"].max()]

In [ ]:
df.drop(df[df["fare_amount"]==df["fare_amount"].min()].index, axis=0,inplace=True)

In [ ]:
df[df["fare_amount"]==df["fare_amount"].min()]

In [ ]:
df.groupby("year")["fare_amount"].mean()


In [ ]:
sns.barplot(df,x=df["year"],y=df["fare_amount"])
plt.title("Year by Average Fare Amount")

In [ ]:
df.columns

In [ ]:
df.drop(["my distance"] , axis=1,inplace=True)

In [ ]:
df.sample(3)

In [ ]:
df[df["distance"]>50]

In [ ]:
df[df["distance"] > 50]["fare_amount"].describe()

In [ ]:
df.drop(df[df["distance"] > 50].index,axis=0,inplace=True)

In [ ]:
df.describe()

In [ ]:
df[df["distance"]<0.1]

In [ ]:
df[df["distance"] <0.1]["fare_amount"].describe()

In [ ]:
df.drop(df[df["distance"]<0.1].index,axis=0,inplace=True)

In [ ]:
df[["distance"]].describe()

In [ ]:
df.info()

In [ ]:
df[df["fare_amount"]>100]["distance"].describe()

In [ ]:
df[(df["fare_amount"]>100 ) & ( df["distance"]<10)]

In [ ]:
df.drop(df[(df["fare_amount"]>100 ) & ( df["distance"]<10)].index,axis=0,inplace=True)

In [ ]:
df[df["fare_amount"]>100]["distance"].describe()

In [ ]:
df[(df["fare_amount"]<10 ) & ( df["distance"]>20)]

In [ ]:
df[(df["fare_amount"]<10 ) & ( df["distance"]>20)].describe()

In [ ]:
df[df["fare_amount"]<10]["distance"].describe()

In [ ]:
df.drop(df[(df["fare_amount"]<10 ) & ( df["distance"]>20)].index,axis=0,inplace=True)

In [ ]:
df[df["fare_amount"]<10]["distance"].describe()

In [ ]:
sns.scatterplot(df,x=df["distance"],y=df["fare_amount"])
plt.title("Fare Amount By Distance ")

In [ ]:
df.head()

In [ ]:
df.drop(["nyc_dist","jfk_dist","lga_dist","ewr_dist","bearing","min"],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df["fare_per_km"]=df["fare_amount"]/df["distance"]

In [ ]:
df.head()

In [ ]:
df.groupby("year")[["fare_per_km"]].mean()

In [ ]:
df[df['fare_per_km']>20]

In [ ]:
df.drop(df[df['fare_per_km']>20].index,axis=0,inplace=True)

In [ ]:
df["fare_per_km"].describe()

In [ ]:
df[df['fare_per_km']<1]

In [ ]:
df.drop(df[df['fare_per_km']<1].index,axis=0,inplace=True)

In [ ]:
df["fare_per_km"].describe()

In [ ]:
df[df['fare_per_km']>15]

In [ ]:
df.drop(df[df['fare_per_km']>15].index,axis=0,inplace=True)

In [ ]:
df["fare_per_km"].describe()

In [ ]:
df.head()

In [ ]:
sns.scatterplot(df,x=df["distance"],y=df["fare_per_km"])

In [ ]:
df["sol_dist"].describe()

In [ ]:
df[df["sol_dist"]>50]

In [ ]:
df.drop(df[df["sol_dist"]>50].index,axis=0,inplace=True)

In [ ]:
df["sol_dist"].describe()

In [ ]:
sns.countplot(data=df, x="hour",palette="viridis",hue="hour",legend=False)
plt.title("Number of Registrations by Hour")
plt.xlabel("Hour of the Day")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
sns.barplot(data=df, x="hour", y="fare_amount", estimator="mean", errorbar=None,palette="viridis",hue="hour",legend=False)

plt.title("Average Fare Amount by Hour")
plt.xlabel("Hour of the Day")
plt.ylabel("Average Fare Amount")
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.countplot(data=df, x="day",palette="viridis",hue="day",legend=False)
plt.title("Number of Registrations by day")
plt.xlabel("Days")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
sns.barplot(data=df, x="day", y="fare_amount", estimator="mean", errorbar=None,palette="viridis",hue="day",legend=False)

plt.title("Average Fare Amount by day")
plt.xlabel("Days")
plt.ylabel("Average Fare Amount")
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.countplot(data=df, x="month",palette="viridis",hue="month",legend=False)
plt.title("Number of Registrations by month")
plt.xlabel("months")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
sns.barplot(data=df, x="month", y="fare_amount", estimator="mean", errorbar=None,palette="viridis",hue="month",legend=False)

plt.title("Average Fare Amount by month")
plt.xlabel("month")
plt.ylabel("Average Fare Amount")
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.countplot(data=df, x="year",palette="viridis",hue="year",legend=False)
plt.title("Number of Registrations by year")
plt.xlabel("year ")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
sns.barplot(data=df, x="year", y="fare_amount", estimator="mean", errorbar=None,palette="viridis",hue="year",legend=False)

plt.title("Average Fare Amount by year")
plt.xlabel("year")
plt.ylabel("Average Fare Amount")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df.head()

In [ ]:
df["Car Condition"].unique()

In [ ]:
carmap={
    "Bad":0,
    "Good":1,
    "Very Good":2,
    "Excellent":3
}
df["Car Condition"]=df["Car Condition"].map(carmap)

In [ ]:
df.columns=df.columns.str.strip().str.lower().str.replace(" ","_")

In [ ]:
df.columns

In [ ]:
df = pd.get_dummies(df, columns=["weather", "traffic_condition"], drop_first=True)

In [ ]:
df.head()

In [ ]:
x=df.drop(df[["fare_amount","fare_per_km"]],axis=1)
y=df["fare_amount"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
scaler=RobustScaler()

In [ ]:
X_train[["sol_dist", "distance"]] = scaler.fit_transform(X_train[["sol_dist", "distance"]])
X_test[["sol_dist", "distance"]] = scaler.transform(X_test[["sol_dist", "distance"]])  

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "Bagging": BaggingRegressor(),
    "XGBoost": xgb.XGBRegressor(),
    "KNN": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = r2_score(y_train, y_train_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_rmse = np.sqrt(train_mse)
    train_mae = mean_absolute_error(y_train, y_train_pred)

    test_r2 = r2_score(y_test, y_test_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    print(f"Model: {name}")
    print(f"Train - R2: {train_r2:.4f} | RMSE: {train_rmse:.4f} | MSE: {train_mse:.4f} | MAE: {train_mae:.4f}")
    print(f"Test  - R2: {test_r2:.4f} | RMSE: {test_rmse:.4f} | MSE: {test_mse:.4f} | MAE: {test_mae:.4f}")
    print("-" * 50)

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_model = XGBRegressor()
search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1
)

search.fit(X_train, y_train)
print("Best parameters:", search.best_params_)
print("Best CV score (R2):", search.best_score_)


In [ ]:
best_model = search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = search.best_estimator_.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f" Mean Squared Error (MSE): {mse:.4f}")
print(f" Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f" Mean Absolute Error (MAE): {mae:.4f}")
print(f" R² Score: {r2:.4f}")


In [ ]:
pickle.dump(best_model, open("model.pkl", "wb"))
pickle.dump(scaler, open("scaler.pkl", "wb"))


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import pickle

st.title("Taxi Fare Prediction")
st.write("This app predicts the taxi fare amount based on various trip and environment details.")
st.image("https://oceansidetaxi.ca/wp-content/uploads/uber-estimate.png", width=700)

# --- User Inputs ---
car_condition = st.selectbox("Car Condition", ["Bad", "Good", "Very Good", "Excellent"])
car_map = {"Bad": 0, "Good": 1, "Very Good": 2, "Excellent": 3}
car_condition_val = car_map[car_condition]

passenger_count = st.number_input("Number of Passengers", min_value=0, max_value=6, value=1)

hour = st.slider("Hour of the Day", 0, 23, 12)
day = st.slider("Day of Month", 1, 31, 15)
month = st.selectbox("Month", list(range(1, 13)))
weekday_name = st.selectbox("Day of the Week", [
    "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
])
weekday_map = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
}
weekday = weekday_map[weekday_name]
year = st.selectbox("Year", [2009,2010,2011,2012,2013,2014,2015])

sol_dist = st.number_input("Distance from statue of liberty (km)", min_value=0.0, max_value=50.0, step=0.1)
distance = st.number_input("Trip Distance (km)", min_value=0.0, max_value=50.0, step=0.1)

# Weather (cloudy is dropped in one-hot, so not shown)
weather = st.selectbox("Weather", ["Cloudy", "Rainy", "Stormy", "Sunny", "Windy"])
weather_features = ['weather_rainy', 'weather_stormy', 'weather_sunny', 'weather_windy']
weather_encoding = [1 if weather.lower() in w else 0 for w in weather_features]

# Traffic (Congested Traffic is dropped)
traffic = st.selectbox("Traffic Condition", ["Congested Traffic", "Dense Traffic", "Flow Traffic"])
traffic_features = ['traffic_condition_Dense Traffic', 'traffic_condition_Flow Traffic']
traffic_encoding = [1 if traffic in t else 0 for t in traffic_features]

# --- Prepare Input ---
input_data = [[
    car_condition_val, passenger_count, hour, day, month, weekday,
    year, sol_dist, distance
] + weather_encoding + traffic_encoding]

columns = [
    'car_condition', 'passenger_count', 'hour', 'day', 'month', 'weekday',
    'year', 'sol_dist', 'distance',
    'weather_rainy', 'weather_stormy', 'weather_sunny', 'weather_windy',
    'traffic_condition_Dense Traffic', 'traffic_condition_Flow Traffic'
]

input_df = pd.DataFrame(input_data, columns=columns)

# --- Load Scaler and Model ---
scaler = pickle.load(open("scaler.pkl", "rb"))
model = pickle.load(open("model.pkl", "rb"))

# --- Scale sol_dist and distance only ---
input_df[["sol_dist", "distance"]] = scaler.transform(input_df[["sol_dist", "distance"]])

# --- Predict ---
if st.button("Predict Fare"):
    predicted_fare = model.predict(input_df)[0]
    st.write("**Predicted Fare Amount:**")
    st.success(f"${predicted_fare:.2f}")
    st.write("---")
    st.write("**Input Data Used for Prediction:**")
    st.dataframe(input_df)


In [ ]:
!streamlit run app.py